In [1]:
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random
import threading
import coinconfig

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.helpers import round_step_size
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [2]:
#0:BTCBUSD, 1:BTCUSDT, 2:DOGEUSDT, 3:DOGEBUSD, 4:ETHBUSD, 5:ETHUSDT, 6:EOSUSDT, 7:LTCUSDT, 8:BNBBUSD

In [3]:
client = Client(KEY, SECRET, testnet=False)
config=coinconfig.tradeconfig
coin=coinconfig.coinlist[0]

In [4]:
symbol=coin["symbol"]
asset=coin["asset"]
recvWindow=config["recvWindow"]
dualSidePosition=config["dualSidePosition"]
marginType=config["marginType"]

leverage=coin["leverage"]
roundity=coin["roundity"]
profit_percent=coin["profit_percent"]
diference=coin["diference"]
stop_percent=profit_percent-diference
balance_part=coin["balance_part"]
compensation=coin["compensation"]
ingebit=coin["ingebit"]

In [5]:
client.futures_change_leverage(symbol=symbol, leverage=leverage, recvWindow=recvWindow)

{'symbol': 'BTCBUSD', 'leverage': 50, 'maxNotionalValue': '50000'}

In [6]:
# Margin type
def margin_type_to_isolated():
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    body=positions[positions['symbol']==symbol]['isolated'].astype(bool).tolist()[0]

    if body==True:
        client.futures_change_margin_type(symbol=symbol, marginType=marginType, recvWindow=recvWindow)
        print(f'Changed to {marginType}')
    else:
        print(f'Already {marginType}')

In [7]:
margin_type_to_isolated()

Already CROSSED


In [8]:
# Hedge mode turn on

def hedge_mode_turn_on():
    status=client.futures_get_position_mode() 
    if status['dualSidePosition']==False:
        client.futures_change_position_mode(dualSidePosition=dualSidePosition, recvWindow=recvWindow)
        client.futures_get_position_mode()
        print('Dual turned on')
    else:
        print('Already Dual')
    return(status['dualSidePosition'])

In [9]:
hedge_mode_turn_on()

Already Dual


True

In [10]:
def get_assetbalance(symbol):
    status = client.futures_account()
    balance = pd.DataFrame(status['assets'])
    assetbalance = float(balance.loc[balance['asset'] == asset, 'availableBalance'])
    return(assetbalance)

In [11]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    position_long=float(open_positions[open_positions['positionSide']=='LONG']['entryPrice'])
    position_short=float(open_positions[open_positions['positionSide']=='SHORT']['entryPrice'])
    return(position_long,position_short)

In [12]:
get_opened_positions(symbol)

(16574.4, 16574.3)

In [13]:
def get_quant(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    open_positions=positions[positions['symbol']==symbol]
    quant=float(open_positions[open_positions['positionSide']=='LONG']['positionAmt'])
    if quant==0:
        quant=abs(float(open_positions[open_positions['positionSide']=='SHORT']['positionAmt']))
    return(quant)

In [14]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [15]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [16]:
get_symbol_price(symbol)

16586.4

In [17]:
#получить tick size

def get_tick_size(symbol):
    data = client.futures_exchange_info()
    tick_size = 0.0
    the_symbol = symbol

    found = False
    info = data['symbols']
    for s in range(len(info)):
        if info[s]['symbol'] == the_symbol:
            filters = info[s]['filters']
            for f in range(len(filters)):
                if filters[f]['filterType'] == 'PRICE_FILTER':
                    tick_size = float(filters[f]['tickSize'])
                    found = True
                    break
            break
    return(tick_size)

In [18]:
#Trade config

# maxposition=0.016 #1 = 100% = 1 BTC,
asset_balance=get_assetbalance(symbol)
symbol_price=get_symbol_price(symbol)
maxposition=round(asset_balance*balance_part*leverage/symbol_price,3)
quantity=str(round(maxposition/2,roundity))
tick_size=get_tick_size(symbol)
symbolround=abs(str(tick_size).find('.') - len(str(tick_size))) - 1
quantity

'0.001'

In [19]:
#Stop loss LONG 
def open_sl_long(symbol,quant,tick_size,stop_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        stop_price_trigger=str(round(float(entry_price)*(1-stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }

        open_position_sl_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_sl_opened=float(open_position_sl_long_response[0]['price'])
        print('LONG sl order opened price:'+str(long_sl_opened))
        return(long_sl_opened)
    
    except :
        print('\n\nSL long not open...')
        

In [20]:
def open_sl_long_steps(symbol,quantity,tick_size,stop_percent):
    stop_step=stop_percent/5  
    quant=str(get_quant(symbol))
    x = threading.Thread(target=open_sl_long, args=(symbol,str(round(float(quant)/(ingebit*2),roundity)),tick_size,stop_step*3/2))
    x.start()
    y = threading.Thread(target=open_sl_long, args=(symbol,str(round(float(quant)/(ingebit),roundity)),tick_size,stop_step*2))
    y.start()
    z = threading.Thread(target=open_sl_long, args=(symbol,str(round(float(quant)/(ingebit*2),roundity)),tick_size,stop_step*5/2))
    z.start()
    w = threading.Thread(target=open_sl_long, args=(symbol,quant,tick_size,stop_step*3))
    w.start()

In [21]:
#Take Profit LONG 

def open_tp_long(symbol,quant,tick_size,profit_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[0]
        take_price_trigger=str(round(float(entry_price)*(1+profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*0.9999, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "positionSide": "LONG",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }
        open_position_tp_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        long_tp_opened=float(open_position_tp_long_response[0]['price'])
        print('LONG tp order opened price:'+str(long_tp_opened))
        return(long_tp_opened)
        
    except :
        print('\n\nTP long not open...')

In [22]:
def open_tp_long_steps(symbol,quantity,tick_size,profit_percent):
    quant=str(round(float(quantity)/(ingebit*1.5),roundity))
    profit_step=profit_percent/5
    
    # x = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size,profit_percent))
    # x.start()
    # y = threading.Thread(target=open_tp_long, args=(symbol,quant,tick_size,profit_step))
    # y.start()
    z = threading.Thread(target=open_tp_long, args=(symbol,quant,tick_size,profit_step*2))
    z.start()
    w = threading.Thread(target=open_tp_long, args=(symbol,quantity,tick_size,profit_step*3))
    w.start()

In [23]:
#Открыть sl&tp LONG если есть позиция

def long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent):
    position=get_opened_positions(symbol)
    if position[0]>0:
        x = threading.Thread(target=open_tp_long_steps, args=(symbol,quantity,tick_size,profit_percent))
        x.start()
        y = threading.Thread(target=open_sl_long_steps, args=(symbol,quantity,tick_size,stop_percent))
        y.start()

In [24]:
#Открыть позицию LONG 

def open_position_long(symbol,quantity,tick_size,profit_percent,stop_percent,close_price):
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "BUY",
                "positionSide": "LONG",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_long_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    long_opened=float(open_position_long_response[0]['price'])
    if long_opened>0:
        print('LONG order opened price:'+str(long_opened))
        long_tp_sl=long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    else:
        print('LONG not open')

In [25]:
def open_position_long_steps(symbol,profit_percent):
    quant=str(round(float(quantity)/ingebit,roundity))
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1+0.001),symbolround))
    open_position_long(symbol,quantity,tick_size,profit_percent,stop_percent,close_price)

In [26]:
#Stop loss SHORT 

def open_sl_short(symbol,quant,tick_size,stop_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        stop_price_trigger=str(round(float(entry_price)*(1+stop_percent), symbolround))
        stop_price=str(round(float(stop_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "STOP",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": stop_price_trigger,
                    "price": stop_price
                }
            ]
        }
        
        open_position_sl_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_sl_opened=float(open_position_sl_short_response[0]['price'])
        print('SHORT sl order opened price:'+str(short_sl_opened))
        return(short_sl_opened)
    
    except :
        print('\n\nSL short not open...')


In [27]:
def open_sl_short_steps(symbol,quantity,tick_size,stop_percent):
    stop_step=stop_percent/5 
    quant=str(get_quant(symbol))
    x = threading.Thread(target=open_sl_long, args=(symbol,str(round(float(quant)/(ingebit*2),roundity)),tick_size,stop_step*3/2))
    x.start()
    y = threading.Thread(target=open_sl_short, args=(symbol,str(round(float(quant)/(ingebit),roundity)),tick_size,stop_step*2))
    y.start()
    z = threading.Thread(target=open_sl_short, args=(symbol,str(round(float(quant)/(ingebit*2),roundity)),tick_size,stop_step*5/2))
    z.start()
    w = threading.Thread(target=open_sl_short, args=(symbol,quant,tick_size,stop_step*3))
    w.start()

In [28]:
#Take Profit SHORT

def open_tp_short(symbol,quant,tick_size,profit_percent):
    try:
        positions=get_opened_positions(symbol)
        entry_price=positions[1]
        take_price_trigger=str(round(float(entry_price)*(1-profit_percent), symbolround))
        take_price=str(round(float(take_price_trigger)*1.0001, symbolround))
        tick=str(tick_size)
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "positionSide": "SHORT",
                    "type": "TAKE_PROFIT",
                    "quantity": quant,
                    "timeInForce":"GTE_GTC",
                    "tickSize": tick,
                    "stopPrice": take_price_trigger,
                    "price": take_price
                }
            ]
        }

        open_position_tp_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
        short_tp_opened=float(open_position_tp_short_response[0]['price'])
        print('SHORT tp order opened price:'+str(short_tp_opened))
        return(short_tp_opened)
    
    except :
        print('\n\nTp short not open...')


In [29]:
def open_tp_short_steps(symbol,quantity,tick_size,profit_percent):
    quant=str(round(float(quantity)/3,roundity))
    profit_step=profit_percent/5
    
    # x = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size,profit_percent))
    # x.start()
    # y = threading.Thread(target=open_tp_short, args=(symbol,quant,tick_size,profit_step))
    # y.start()
    z = threading.Thread(target=open_tp_short, args=(symbol,quant,tick_size,profit_step*2))
    z.start()
    w = threading.Thread(target=open_tp_short, args=(symbol,quantity,tick_size,profit_step*3))
    w.start()

In [30]:
#Открыть sl&tp SHORT если есть позиция
       
def short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent):
    position=get_opened_positions(symbol)
    if position[1]>0:
        x = threading.Thread(target=open_tp_short_steps, args=(symbol,quantity,tick_size,profit_percent))
        x.start()
        y = threading.Thread(target=open_sl_short_steps, args=(symbol,quantity,tick_size,stop_percent))
        y.start()

In [31]:
#Открыть позицию SHORT

def open_position_short(symbol,quantity,tick_size,profit_percent,stop_percent,close_price):
    tick=str(tick_size)
    params = {
        "batchOrders": [
            {
                "symbol":symbol,
                "side": "SELL",
                "positionSide": "SHORT",
                "type": "LIMIT",
                "quantity": quantity,
                "timeInForce":"GTC",
                "tickSize": tick,
                "price": close_price        
            }
        ]
    }
    open_position_short_response=send_signed_request('POST', '/fapi/v1/batchOrders', params)
    short_opened=float(open_position_short_response[0]['price'])    
    if short_opened>0:
        print('SHORT order opened price:'+str(short_opened))
        short_tp_sl=short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    else:
        print('SHORT not open')

In [32]:
def open_position_short_steps(symbol,profit_percent):
    quant=str(round(float(quantity)/ingebit,roundity))
    sprice=get_symbol_price(symbol)
    close_price=str(round(sprice*(1-0.001),symbolround))
    open_position_short(symbol,quantity,tick_size,profit_percent,stop_percent,close_price)

In [33]:
def get_profit(symbol):
    trades = client.futures_account_trades(symbol=symbol)
    dataset=pd.DataFrame(trades).iloc[-10:]
    bnbcom=dataset[dataset['commissionAsset']=='BNB']['commission'].astype(float).sum()
    busdcom=dataset[dataset['commissionAsset']==asset]['commission'].astype(float).sum()
    com=(bnbcom*270+busdcom)*-1
    real=dataset['realizedPnl'].astype(float).sum()
    profit=float(com+real)
    dataseto=pd.DataFrame(trades)['realizedPnl'].astype(float)[-50:]
    dataseto=pd.DataFrame(dataseto)
    datasetneg=len(dataseto[dataseto['realizedPnl']<0].index)
    datasetpos=len(dataseto[dataseto['realizedPnl']>0].index)
    print(profit,'+',datasetpos,'-',datasetneg)

In [34]:
get_profit(symbol)

-0.0737061 + 4 - 6


In [35]:
def get_direction(symbol):
    trades = client.futures_account_trades(symbol=symbol)[-1:]
    dataset = pd.DataFrame(trades)
    resdata = float(dataset['realizedPnl'])
    direction=1
    if resdata < 0:
        direction=0
    return(direction)

In [36]:
get_direction(symbol)

1

In [37]:
def changer(symbol):
    direction=get_direction(symbol)
    f = open("directdoc.txt", "r")
    direc=f.read()
    if direction==0: #реверс
        if direc=='0':
            x = threading.Thread(target=open_position_long_steps, args=(symbol,profit_percent*1.5))
            x.start()
            y = threading.Thread(target=open_position_short_steps, args=(symbol,profit_percent))
            y.start()
            print('Direction switched from SHORT to LONG')
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()
        else:
            x = threading.Thread(target=open_position_long_steps, args=(symbol,profit_percent))
            x.start()
            y = threading.Thread(target=open_position_short_steps, args=(symbol,profit_percent*1.5))
            y.start()
            print('Direction switched from LONG to SHORT')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
    else: 
        if direc=='0':
            x = threading.Thread(target=open_position_long_steps, args=(symbol,profit_percent))
            x.start()
            y = threading.Thread(target=open_position_short_steps, args=(symbol,profit_percent*1.5))
            y.start()
            print('Direction SHORT again')
            f = open("directdoc.txt", "w")
            f.write("0")
            f.close()
        if direc=='1':
            x = threading.Thread(target=open_position_long_steps, args=(symbol,profit_percent*1.5))
            x.start()
            y = threading.Thread(target=open_position_short_steps, args=(symbol,profit_percent))
            y.start()
            print('Direction LONG again')
            f = open("directdoc.txt", "w")
            f.write("1")
            f.close()

In [38]:
def check_positions_new(symbol):
    positions=get_opened_positions(symbol)
    orders=client.futures_get_open_orders(symbol=symbol)
    orders_df=pd.DataFrame(orders)
    if positions[0]+positions[1]==0:
        if orders_df.empty:
            changer(symbol)
            
    orders_tp_number=len(orders_df[orders_df['origType']=='TAKE_PROFIT'])
    if positions[0]>0 and (orders_tp_number==1 or orders_tp_number==3):
        sprice=get_symbol_price(symbol)
        if sprice<(positions[0])*(1+profit_percent/10):
            quant=str(round(get_quant(symbol),roundity))
            params={ 
                "batchOrders": [
                    {
                        "symbol":symbol, 
                        "positionSide": "LONG", 
                        "side":'SELL', 
                        "type":'MARKET',
                        "quantity":quant
                    }
                ]
            }
            one_stop=send_signed_request('POST', '/fapi/v1/batchOrders', params)
            if str(float(one_stop[0]['origQty']))==quant:
                print(f'Compensation LONG price {sprice}')

    if positions[1]>0 and (orders_tp_number==1 or orders_tp_number==3):
        sprice=get_symbol_price(symbol)
        if sprice>(positions[1])*(1-profit_percent/10):
            quant=str(round(get_quant(symbol),roundity))
            params={ 
                "batchOrders": [
                    {
                        "symbol":symbol, 
                        "positionSide": "SHORT", 
                        "side":'BUY', 
                        "type":'MARKET',
                        "quantity":quant
                    }
                ]
            }
            one_stop=send_signed_request('POST', '/fapi/v1/batchOrders', params)
            if str(float(one_stop[0]['origQty']))==quant:
                print(f'Compensation SHORT price {sprice}')
#     else:

#         if positions[0]>0 and positions[1]==0:
#             quant=str(round(get_quant(symbol),roundity))
#             sprice=get_symbol_price(symbol)
#             asset=get_assetbalance(symbol)
#             if (asset*leverage)/(sprice*ingebit*2)>float(quant):
#                 if sprice<positions[0]*(1-profit_percent/4):
#                     sprice=get_symbol_price(symbol)
#                     close_price=str(round(sprice*(1-0.001),symbolround))                
#                     open_position_short(symbol,quant,tick_size,profit_percent,stop_percent,close_price)
#                     print('SHORT compensation opened')
#                     # client.futures_cancel_all_open_orders(symbol=symbol)
#                     # long_tp_sl_opening(symbol,quant,profit_percent,stop_percent)
#                 else:
#                     print('SHORT compensation not in condition')
#         if positions[1]>0 and positions[0]==0:
#             quant=str(round(get_quant(symbol),roundity))
#             sprice=get_symbol_price(symbol)
#             asset=get_assetbalance(symbol)
#             if (asset*leverage)/(sprice*ingebit*2)>float(quant):
#                 if sprice>positions[1]*(1+profit_percent/4):
#                     sprice=get_symbol_price(symbol)
#                     close_price=str(round(sprice*(1+0.001),symbolround))                
#                     open_position_long(symbol,quant,tick_size,profit_percent,stop_percent,close_price)
#                     print('LONG compensation opened')
#                     # client.futures_cancel_all_open_orders(symbol=symbol)
#                     # short_tp_sl_opening(symbol,quant,profit_percent,stop_percent)
#                 else:
#                     print('LONG compensation not in condition')

In [39]:
def check_step(symbol):
    quant=str(get_quant(symbol)*2)
    sprice=get_symbol_price(symbol)
    asset=get_assetbalance(symbol)
    # if (asset*leverage)/(sprice*ingebit*2)>float(quant):
    positions=get_opened_positions(symbol)
    if positions[0]>0:
        if sprice>(positions[0])*(1+profit_percent/1.9):
            sprice=get_symbol_price(symbol)
            close_price=str(round(sprice*(1+0.001),symbolround))
            open_position_long(symbol,quant,tick_size,profit_percent,stop_percent,close_price)
            print('LONG addinational 1')
            # client.futures_cancel_all_open_orders(symbol=symbol)
            # short_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)

    if positions[1]>0:
        if sprice<(positions[1])*(1-profit_percent/1.9):
            sprice=get_symbol_price(symbol)
            close_price=str(round(sprice*(1-0.001),symbolround))
            open_position_short(symbol,quant,tick_size,profit_percent,stop_percent,close_price)
            print('SHORT addinational 1')
            # client.futures_cancel_all_open_orders(symbol=symbol)
            # long_tp_sl_opening(symbol,quantity,profit_percent,stop_percent)
    else:
        print('No positions')

In [40]:
def main(step):
    try:
        x = threading.Thread(target=check_positions_new(symbol))
        x.start()
        y = threading.Thread(target=get_profit(symbol))
        y.start()
        z = threading.Thread(target=check_step(symbol))
        z.start()
    except :
        print('\n\nSomething went wrong. Continuing...')

In [41]:
def start_stop():
    starttime=time.time()
    counterr=1
    timeout = time.time() + 60*60*6  # 60 seconds times 60 meaning the script will run for 12 hr
    while time.time() <= timeout:
        try:

            print("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
            main(counterr)
            counterr=counterr+1
            if counterr>3:
                counterr=1
            time.sleep(4 - ((time.time() - starttime) % 4.0)) # 1 minute interval between each new execution
        except KeyboardInterrupt:
            print('\n\KeyboardInterrupt. Stopping.')
        

In [ ]:
start_stop()

script continue running at 2022-11-26 13:30:37
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:30:45
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:30:49
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:30:53
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:30:57
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:01
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:05
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:09
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:13
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:17
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:21
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:25
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:29
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:33
-0.0737061 + 4 - 6
script continue running at 2022-11-26 13:31:37
-0.0737061 + 4 - 6
script con